In [ ]:
!git clone https://github.com/nateraw/huggingface-vit-finetune.git
!pip install adversarial-robustness-toolbox
!pip install timm
%cd huggingface-vit-finetune/
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/transformers.git@master --upgrade

Cloning into 'huggingface-vit-finetune'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 10), reused 19 (delta 6), pack-reused 0
Unpacking objects: 100% (24/24), done.
     |████████████████████████████████| 962kB 8.0MB/s 
     |████████████████████████████████| 6.8MB 27.8MB/s 
     |████████████████████████████████| 3.2MB 48.5MB/s 
     |████████████████████████████████| 235kB 48.5MB/s 
     |████████████████████████████████| 21.6MB 1.5MB/s 
     |████████████████████████████████| 25.3MB 97kB/s 
     |████████████████████████████████| 747kB 51.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import art
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier

from transformers import ViTFeatureExtractor, ViTForImageClassification
import requests

import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar_trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=64, shuffle=True, num_workers=2)
cifar_testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=64, shuffle=False, num_workers=2)
cifar_classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10', return_dict=False)
model.to(device)
model = nn.Sequential(
    model,
    nn.Identity()
)
model

Sequential(
  (0): ViTForImageClassification(
    (vit): ViTModel(
      (embeddings): ViTEmbeddings(
        (patch_embeddings): PatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): ViTEncoder(
        (layer): ModuleList(
          (0): ViTLayer(
            (attention): ViTAttention(
              (attention): ViTSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (intermediate): ViTI

In [ ]:
# cifar-10 DATASET consists of 60000 images
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs[0], 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %d %%' % (total, 100 * correct / total))


print('Accuracy of the network on %d cifar-10 test images: %d %%' % (
    total,
    100 * correct / total))

Accuracy of the network on 64 cifar-10 test images: 98 %
Accuracy of the network on 704 cifar-10 test images: 99 %


KeyboardInterrupt: ignored

In [ ]:
outputs[0].shape

torch.Size([64, 10])

In [ ]:
class Transformer(nn.Module):
  def __init__(self, model):
    super(Transformer, self).__init__()
    self.model = model
  def forward(self, x):
    return model(x)[0]
Model = Transformer(model.cpu())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
classifier = PyTorchClassifier(
    model=Model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=10,
)

In [ ]:
eps = 0.2

In [ ]:
attack_FGM = FastGradientMethod(estimator=classifier, eps=eps)
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_testloader):
  inputs, labels = inputs.cpu(), labels.numpy()
  x_test_adv = attack_FGM.generate(x=inputs)
  outputs = classifier.predict(x_test_adv)
  predicted = np.argmax(outputs, 1)
  total += labels.size
  correct += (predicted == labels).sum().item()
  if(i % 10 == 0):
    print('Accuracy of the network on %d cifar-10 test images: %d %%' % (total, 100 * correct / total))


print('Accuracy of the network on %d cifar-10 test images: %d %%' % (
    total,
    100 * correct / total))

Accuracy of the network on 64 cifar-10 test images: 18 %
Accuracy of the network on 704 cifar-10 test images: 18 %
Accuracy of the network on 1344 cifar-10 test images: 19 %
Accuracy of the network on 1984 cifar-10 test images: 19 %
Accuracy of the network on 2624 cifar-10 test images: 18 %
Accuracy of the network on 3264 cifar-10 test images: 19 %
Accuracy of the network on 3904 cifar-10 test images: 19 %
Accuracy of the network on 4544 cifar-10 test images: 19 %
Accuracy of the network on 5184 cifar-10 test images: 19 %
Accuracy of the network on 5824 cifar-10 test images: 19 %
Accuracy of the network on 6464 cifar-10 test images: 19 %
Accuracy of the network on 7104 cifar-10 test images: 19 %
Accuracy of the network on 7744 cifar-10 test images: 19 %
Accuracy of the network on 8384 cifar-10 test images: 19 %
Accuracy of the network on 9024 cifar-10 test images: 19 %
Accuracy of the network on 9664 cifar-10 test images: 19 %
Accuracy of the network on 10000 cifar-10 test images: 19 %

In [ ]:
attack_PGD_20 = ProjectedGradientDescentPyTorch(
        estimator=classifier,
        norm="inf",
        eps=eps,
        targeted=False,
        eps_step=0.05,
        max_iter=20
)
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_testloader):
  inputs, labels = inputs.cpu(), labels.numpy()
  x_test_adv = attack_PGD_20.generate(x=inputs)
  outputs = classifier.predict(x_test_adv)
  predicted = np.argmax(outputs, 1)
  total += labels.size
  correct += (predicted == labels).sum().item()
  if(i % 10 == 0):
    print('Accuracy of the network on %d cifar-10 test images: %d %%' % (total, 100 * correct / total))


print('Accuracy of the network on %d cifar-10 test images: %d %%' % (
    total,
    100 * correct / total))

/usr/local/lib/python3.7/dist-packages/art/estimators/classification/pytorch.py:670: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_grad = torch.tensor(x).to(self._device)
/usr/local/lib/python3.7/dist-packages/art/estimators/classification/pytorch.py:671: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_grad = torch.tensor(y).to(self._device)


Accuracy of the network on 64 cifar-10 test images: 1 %


Accuracy of the network on 704 cifar-10 test images: 0 %


Accuracy of the network on 1344 cifar-10 test images: 0 %


Accuracy of the network on 1984 cifar-10 test images: 0 %


...

Accuracy of the network on 7744 cifar-10 test images: 0 %


Accuracy of the network on 8384 cifar-10 test images: 0 %


Accuracy of the network on 9024 cifar-10 test images: 0 %


Accuracy of the network on 9664 cifar-10 test images: 0 %


Accuracy of the network on 10000 cifar-10 test images: 0 %
